In [21]:
!pip install beautifulsoup4
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime

In [23]:
# extracción de datos
url = "https://www.estesparkweather.net/archive_reports.php?date=202005" # direccion de la extracción de datos
response = requests.get(url) # solicitud para datos
print(response) # imprimir respuesta
print(response.content[:200]) # vista de los primeros 200 caracteres de datos
sopa =  BeautifulSoup(response.content,'html.parser') # analizar el contenido de la pagina
tabla = sopa.find_all('table') # encontrar los datos en la etiqueta html table

<Response [200]>
b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<!-- DW6 -->\n<head>\n<!-- Copyright 2005 Macrom'


In [24]:
sopa =  BeautifulSoup(response.content,'html.parser') # analizar el contenido de la pagina
tabla = sopa.find_all('table') # encontrar los datos en la etiqueta html table
print(tabla[0]) # imprimir la primera parte de datos puesto que se comienza en el cero

<table border="0" cellpadding="3" cellspacing="0">
<tr class="table-top">
<td colspan="2">May 1 Average and Extremes</td>
</tr>
<tr class="column-light">
<td>Average temperature</td><td> 57.8°F</td>
</tr>
<tr class="column-dark">
<td>Average humidity</td><td> 45%</td>
</tr>
<tr class="column-light">
<td>Average dewpoint</td><td> 33.9°F</td>
</tr>
<tr class="column-dark">
<td>Average barometer</td><td> 29.6 in.</td>
</tr>
<tr class="column-light">
<td>Average windspeed</td><td> 5.2 mph</td>
</tr>
<tr class="column-dark">
<td>Average gustspeed</td><td> 8.0 mph</td>
</tr>
<tr class="column-light">
<td>Average direction</td><td> 238° (WSW)</td>
</tr>
<tr class="column-dark">
<td>Rainfall for month</td><td> 0.00 in.</td>
</tr>
<tr class="column-light">
<td>Rainfall for year</td><td> 2.40 in.</td>
</tr>
<tr class="column-dark">
<td>Maximum rain per minute</td><td> 0.00 in. on day 01 at time 00:00</td>
</tr>
<tr class="column-light">
<td>Maximum temperature</td><td> 71.8°F on day 01 at time 1

In [25]:
tabla[0].text # obtener los datos de la tabla en formato de texto

'\n\nMay 1 Average and Extremes\n\n\nAverage temperature 57.8°F\n\n\nAverage humidity 45%\n\n\nAverage dewpoint 33.9°F\n\n\nAverage barometer 29.6 in.\n\n\nAverage windspeed 5.2 mph\n\n\nAverage gustspeed 8.0 mph\n\n\nAverage direction 238° (WSW)\n\n\nRainfall for month 0.00 in.\n\n\nRainfall for year 2.40 in.\n\n\nMaximum rain per minute 0.00 in. on day 01 at time 00:00\n\n\nMaximum temperature 71.8°F on day 01 at time 14:41\n\n\nMinimum temperature 41.5°F on day 01 at time 06:24\n\n\nMaximum humidity 83% on day 01 at time 00:00\n\n\nMinimum humidity 19% on day 01 at time 14:07\n\n\nMaximum pressure 29.724 in. on day 01 at time 22:22\n\n\nMinimum pressure 29.515 in. on day 01 at time 16:13\n\n\nMaximum windspeed 18.4 mph on day 01 at time 21:02\n\n\nMaximum gust speed 29.9 mph  from 063 °(ENE) on day 01 at time 20:57\n\n\nMaximum heat index 76.6°F on day 01 at time 15:57\n\n'

In [26]:
list(filter(None, tabla[0].text.splitlines())) # filtrar y quitar los campos vacíos de la lista

['May 1 Average and Extremes',
 'Average temperature 57.8°F',
 'Average humidity 45%',
 'Average dewpoint 33.9°F',
 'Average barometer 29.6 in.',
 'Average windspeed 5.2 mph',
 'Average gustspeed 8.0 mph',
 'Average direction 238° (WSW)',
 'Rainfall for month 0.00 in.',
 'Rainfall for year 2.40 in.',
 'Maximum rain per minute 0.00 in. on day 01 at time 00:00',
 'Maximum temperature 71.8°F on day 01 at time 14:41',
 'Minimum temperature 41.5°F on day 01 at time 06:24',
 'Maximum humidity 83% on day 01 at time 00:00',
 'Minimum humidity 19% on day 01 at time 14:07',
 'Maximum pressure 29.724 in. on day 01 at time 22:22',
 'Minimum pressure 29.515 in. on day 01 at time 16:13',
 'Maximum windspeed 18.4 mph on day 01 at time 21:02',
 'Maximum gust speed 29.9 mph  from 063 °(ENE) on day 01 at time 20:57',
 'Maximum heat index 76.6°F on day 01 at time 15:57']

In [27]:
outer_data_list = [] # separar los datos y guardar como lista
for data in tabla[:31]: # recorrer datos de cada dia
    value = list(filter(None, data.text.splitlines())) # obtener valores filtrados sin cadenas vacias    
    outer_data_list.append(value) # anexar dichos valores a la lista
outer_data_list[0] # observar los datos de la lista

['May 1 Average and Extremes',
 'Average temperature 57.8°F',
 'Average humidity 45%',
 'Average dewpoint 33.9°F',
 'Average barometer 29.6 in.',
 'Average windspeed 5.2 mph',
 'Average gustspeed 8.0 mph',
 'Average direction 238° (WSW)',
 'Rainfall for month 0.00 in.',
 'Rainfall for year 2.40 in.',
 'Maximum rain per minute 0.00 in. on day 01 at time 00:00',
 'Maximum temperature 71.8°F on day 01 at time 14:41',
 'Minimum temperature 41.5°F on day 01 at time 06:24',
 'Maximum humidity 83% on day 01 at time 00:00',
 'Minimum humidity 19% on day 01 at time 14:07',
 'Maximum pressure 29.724 in. on day 01 at time 22:22',
 'Minimum pressure 29.515 in. on day 01 at time 16:13',
 'Maximum windspeed 18.4 mph on day 01 at time 21:02',
 'Maximum gust speed 29.9 mph  from 063 °(ENE) on day 01 at time 20:57',
 'Maximum heat index 76.6°F on day 01 at time 15:57']

In [28]:
day_data_list = [] # creación de lista para contener los datos 
for i in outer_data_list: # bucle sobre cada dia
    numeric_values = [] # enlistar para sostener valores numericos
    for day_data in i[1:]: #bucle a través de los datos de cada dia
        d = '.'.join(re.findall("\d+",str(day_data.split()[:5]))) # analizar el texto para obtener digitos
        numeric_values.append(d) # anexar a la lista de los valores numericos de la lista
    day_data_list.append(numeric_values) #anexar a la data
# nombrar las columnas de acurdo al sitio web
columns = ['average_temperature', 'average_humidity', 'average_dewpoint', 'average_barometer', 'average_windspeed', 'average_gustspeed', 'average_direction', 'rainfall_for_month', 'rainfall_for_year', 'maximum_rain_per_minute', 'maximum_temperature', 'minimum_temperature', 'maximum_humidity', 
           'minimum_humidity', 'maximum_pressure', 'minimum_pressure', 'maximum_windspeed', 'maximum_gust_speed', 'maximum_heat_index']
# Create a pandas dataframe with the data
clima = pd.DataFrame(day_data_list, columns = columns) # creación del dataframe de pandas con los datos

In [29]:
clima.head() # comprobar que los datos se encuentren ordenados

,average_temperature,average_humidity,average_dewpoint,average_barometer,average_windspeed,average_gustspeed,average_direction,rainfall_for_month,rainfall_for_year,maximum_rain_per_minute,maximum_temperature,minimum_temperature,maximum_humidity,minimum_humidity,maximum_pressure,minimum_pressure,maximum_windspeed,maximum_gust_speed,maximum_heat_index
0,57.8,45,33.9,29.6,5.2,8.0,238,0.00,2.40,0.00,71.8,41.5,83,19,29.724,29.515,18.4,29.9,76.6
1,47.1,83,42.0,29.7,3.1,5.0,16,0.30,2.70,0.02,57.3,39.9,95,60,29.881,29.666,15.0,19.6,57.3
2,51.6,59,36.4,29.7,4.9,7.5,244,0.33,2.73,0.01,65.4,36.1,93,29,29.876,29.620,19.6,26.5,65.4
3,45.1,58,28.9,29.8,7.6,11.8,248,0.37,2.77,0.01,52.2,38.5,94,28,30.024,29.604,25.3,36.8,52.2
4,48.7,24,12.2,30.0,6.3,9.7,241,0.37,2.77,0.00,62.3,37.9,50,12,30.128,29.942,18.4,26.5,62.3


In [30]:
clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   average_temperature      31 non-null     object
 1   average_humidity         31 non-null     object
 2   average_dewpoint         31 non-null     object
 3   average_barometer        31 non-null     object
 4   average_windspeed        31 non-null     object
 5   average_gustspeed        31 non-null     object
 6   average_direction        31 non-null     object
 7   rainfall_for_month       31 non-null     object
 8   rainfall_for_year        31 non-null     object
 9   maximum_rain_per_minute  31 non-null     object
 10  maximum_temperature      31 non-null     object
 11  minimum_temperature      31 non-null     object
 12  maximum_humidity         31 non-null     object
 13  minimum_humidity         31 non-null     object
 14  maximum_pressure         31 non-null     obj

In [31]:
clima = clima.apply(pd.to_numeric) # convertir todas las columnas a datos numericos

In [32]:
clima.info() # confirmar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   average_temperature      31 non-null     float64
 1   average_humidity         31 non-null     int64  
 2   average_dewpoint         31 non-null     float64
 3   average_barometer        31 non-null     float64
 4   average_windspeed        31 non-null     float64
 5   average_gustspeed        31 non-null     float64
 6   average_direction        31 non-null     int64  
 7   rainfall_for_month       31 non-null     float64
 8   rainfall_for_year        31 non-null     float64
 9   maximum_rain_per_minute  31 non-null     float64
 10  maximum_temperature      31 non-null     float64
 11  minimum_temperature      31 non-null     float64
 12  maximum_humidity         31 non-null     int64  
 13  minimum_humidity         31 non-null     int64  
 14  maximum_pressure         31 

In [33]:
# análisis exploratorio
print("(Filas =", clima.shape[0], "/ Columnas =", clima.shape[1], ")") # dimensiones del dataset

(Filas = 31 / Columnas = 19 )


In [34]:
clima.mean() # calculo de media

average_temperature         50.845161
average_humidity            54.161290
average_dewpoint            31.393548
average_barometer           29.741935
average_windspeed            4.490323
average_gustspeed            7.112903
average_direction          173.161290
rainfall_for_month           0.597742
rainfall_for_year            2.997742
maximum_rain_per_minute      0.004194
maximum_temperature         64.393548
minimum_temperature         36.070968
maximum_humidity            84.774194
minimum_humidity            27.193548
maximum_pressure            29.866613
minimum_pressure            29.652323
maximum_windspeed           17.835484
maximum_gust_speed          25.587097
maximum_heat_index          65.209677
dtype: float64

In [35]:
clima.max() # valores numericos maximos

average_temperature         63.100
average_humidity            84.000
average_dewpoint            44.700
average_barometer           30.000
average_windspeed           10.100
average_gustspeed           15.200
average_direction          290.000
rainfall_for_month           1.000
rainfall_for_year            3.400
maximum_rain_per_minute      0.020
maximum_temperature         79.300
minimum_temperature         49.100
maximum_humidity            97.000
minimum_humidity            66.000
maximum_pressure            30.138
minimum_pressure            30.001
maximum_windspeed           27.600
maximum_gust_speed          36.800
maximum_heat_index          78.600
dtype: float64

In [19]:
clima.min() # valores numericos minimos

average_temperature        36.800
average_humidity           20.000
average_dewpoint           11.600
average_barometer          29.300
average_windspeed           2.000
average_gustspeed           3.600
average_direction           1.000
rainfall_for_month          0.000
rainfall_for_year           2.400
maximum_rain_per_minute     0.000
maximum_temperature        50.400
minimum_temperature        24.500
maximum_humidity           44.000
minimum_humidity            8.000
maximum_pressure           29.483
minimum_pressure           29.193
maximum_windspeed          10.400
maximum_gust_speed         15.000
maximum_heat_index         50.400
dtype: float64

In [20]:
clima.std() # desviacion estandar

average_temperature          7.403370
average_humidity            16.850513
average_dewpoint             8.829871
average_barometer            0.185785
average_windspeed            2.074023
average_gustspeed            2.888453
average_direction          101.907179
rainfall_for_month           0.245882
rainfall_for_year            0.245882
maximum_rain_per_minute      0.005642
maximum_temperature          9.022192
minimum_temperature          5.996009
maximum_humidity            13.435549
minimum_humidity            15.721364
maximum_pressure             0.183888
minimum_pressure             0.192941
maximum_windspeed            4.508033
maximum_gust_speed           6.436860
maximum_heat_index           9.828881
dtype: float64